## Rating csv


In [1]:
import pandas as pd
import datetime
import numpy as  np
from tqdm import tqdm
np.random.seed(1337)
with open('./kaggle/rating_train.csv', 'r') as f:
    ls = f.readlines()[1:]
u_map = {}

dates = []
foods = []
users = []
    

with tqdm(total=len(ls)) as pbar:
    for l in ls:
        date_str, user, food = l.strip().split(',')
        date = datetime.datetime.strptime(date_str, '%Y-%m-%d')
        user, food = int(user), int(food)
        if user not in u_map:
            u_map[user] = []
        u_map[user].append( (date, food) )
        
        dates.append(date)
        foods.append(food)
        users.append(user)
        pbar.update(1)
        

        
food_map = {f:i for i, f in enumerate(set(foods))}
user_map = {u:i for i, u in enumerate(set(users))}
    
        


# normalize datetime
# min_date = min(dates)
# days = [(d-min_date).total_seconds() / (60*60*24) for d in dates]
# mean_days, std_days = np.mean(days), np.std(days)
# def normalize_date(min_date, mean_days, std_days, date):
#     days = (date-min_date).total_seconds() / (60*60*24)
#     return (days-mean_days)/std_days
# with tqdm(total=len(u_map)) as pbar:
#     for _,v in u_map.items():
#         for i, pair in enumerate(v):
#             date, food = pair
#             v[i] = (normalize_date(min_date, mean_days, std_days, date), food)
#         pbar.update(1)
        
# item-user matrix, for ALS




100%|██████████| 2681494/2681494 [00:21<00:00, 127270.80it/s]


In [2]:
import numpy as np
a = []
b = []

for k,v in u_map.items():
    ds = [d for d,f in v]
    
    set_ds = set(ds)
    a.append(len(set_ds))
    b.append(len(ds))
print max(a), min(a),np.mean(a), np.std(a)
print max(b), min(b),np.mean(b), np.std(b)

160 23 95.21165644171779 34.48409857182605
5569 194 1028.1802147239264 599.6642539977294


### Food embedding

In [3]:
### ALS embeddingimport numpy as  np
from scipy.sparse import csr_matrix


with tqdm(total=len(u_map)) as pbar:
    
#     split 1/10 for future
    Y_map = {}
    pairs = []
    history_map = {} # for training and validation
    for user,v in u_map.items():
        history = v
        dates_history = [d for d,f in history]
        min_date, max_date = min(dates_history), max(dates_history)
        date_list = sorted(set(dates_history))
        rest_date_set = set(date_list[-len(date_list)//10:])
        
        
        
        history_map[k] = [d for d,f in history if d not in rest_date_set]
#         calculate the Y
        y = np.zeros([1, len(food_map)])
        for d,f in history:
            if d in rest_date_set:
                y[0, food_map[f]] = 1
            else:
                pair = (food_map[f], user_map[user])
                pairs.append(pair)
        Y_map[user] = y
        pbar.update(1)

        
    rows = []
    cols = []
    pairs = set(pairs)
    for pair in pairs:
        f, u = pair
        rows.append(f)
        cols.append(u)
    data = np.ones([len(rows),])
    item_user_matrix = csr_matrix((data, (rows,cols)), shape=(len(food_map), len(user_map)))
    
print len(Y_map)    
        
    



100%|██████████| 2608/2608 [00:01<00:00, 1489.37it/s]

2608


### Transformer with ALS embedding Training

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.init import xavier_normal_
from constants import D_MODEL,PE_MAT_CACHE


def positional_encoding(pos):
    assert D_MODEL % 2 == 0
    pos = torch.tensor(pos, dtype=torch.float32, requires_grad=False)
    pe = torch.zeros([1,D_MODEL], dtype=torch.float32, requires_grad=False)
    for i in range(D_MODEL//2):
        a = torch.tensor(10000, dtype=torch.float32, requires_grad=False)
        b = torch.tensor(2.*i/float(D_MODEL), dtype=torch.float32, requires_grad=False)
        c = pos / torch.pow(a, b)
        pe[0, 2*i] = torch.sin(c)
        pe[0, 2*i+1] = torch.cos(c)
    return pe
def get_pos_mat(length):
    if length > PE_MAT_CACHE:
        print 'sequence length reach PE_MAT_CACHE. %d ' % length
        ret = torch.cat([positional_encoding(i) for i in range(length)], dim=0).cuda()
        ret.requires_grad = False
        global PE_CACHE_MATRIX
        PE_CACHE_MATRIX = ret
        return ret
    else:
        return PE_CACHE_MATRIX[:length]
def mask_matrix(Q_mask_len, batch, Q_len, K_len):
    assert len(Q_mask_len.shape) == 1
#     ByteTensor
    mask = torch.zeros([batch, Q_len, K_len], dtype=torch.uint8, requires_grad=False)
    for i in range(batch):
        seq_len = Q_mask_len[i]
        if seq_len == Q_len:
            continue
        mask[i,seq_len:,:] = 1
        
    return mask.cuda()

    
PE_CACHE_MATRIX = torch.cat([positional_encoding(i) for i in range(0,PE_MAT_CACHE)], dim=0).cuda()
PE_CACHE_MATRIX.requires_grad = False

# construct neuron network

def scaled_dot_attention(Q, K, V, mask=None):
    assert Q.size()[-1] == K.size()[-1]
    assert len(Q.size()) == 3 and len(K.size()) == 3 and len(V.size()) == 3
    dk = torch.tensor(K.size()[-1], dtype=torch.float32, requires_grad=False).cuda()
    out = torch.matmul(Q,K.permute(0,2,1)) / torch.sqrt(dk) 
    if mask is not None:
        out.masked_fill_(mask, -float('inf'))
#         print 'mask out', out.shape, V.shape, F.softmax(out, dim=-1).shape, torch.matmul(F.softmax(out, dim=-1), V).shape
#         print F.softmax(out, dim=-1)
        
    return torch.matmul(F.softmax(out, dim=1), V)
                            
class Transformer(nn.Module):

    def __init__(self, layer_num, dk, dv, dm, h, p_drop, d_ff):
        super(Transformer, self).__init__()
        
        self.encoder = Stack_Encoder(layer_num, dk, dv, dm, h, p_drop, d_ff)
        self.decoder = Stack_Decoder(layer_num, dk, dv, dm, h, p_drop, d_ff)
        self.emb_drop = nn.Dropout(p_drop)

    def forward(self, Q, K, Q_mask_len):
    
    #         encoder
        batch, K_len, d = K.size()
#         pos matrix will fit the batch size
#         without pos.repeat() is faster
        K = K + get_pos_mat(K_len)
        K = self.emb_drop(K)
        
        en_out = self.encoder(K)
        
#         decoder
        batch, Q_len, d = Q.size()
        
        Q = Q + get_pos_mat(Q_len)
        Q = self.emb_drop(Q)
        
        mask = mask_matrix(Q_mask_len, batch, Q_len, K_len)
        de_out = self.decoder(Q, en_out, mask)
        return de_out
         
        


class Stack_Encoder(nn.Module):
    """
    Stacked Encoder
    """
    def __init__(self, layer_num, dk, dv, dm, h, p_drop, d_ff):
        super(Stack_Encoder, self).__init__()
        self.encoders = nn.ModuleList([Encoder(dk, dv, dm, h, p_drop, d_ff) for i in range(layer_num)])

    def forward(self, K):
        # ModuleList can act as an iterable, or be indexed using ints
        for lay in self.encoders:
            K = lay(K)
        return K         
    
class Encoder(nn.Module):
    def __init__(self, dk, dv, dm, h, p_drop, d_ff):
        super(Encoder, self).__init__()
#         attention residual block
        self.multi_head_attention_layer = Multi_Head_attention_layer(dk, dv, dm, h)
        self.attention_norm_lay = nn.LayerNorm([dm,])
        self.att_drop = nn.Dropout(p_drop)
#         feed forward residual block
        self.fcn = PositionwiseFeedForward(dm, d_ff)
        self.linear_drop = nn.Dropout(p_drop)
        self.ff_norm_lay = nn.LayerNorm([dm, ])
        

    def forward(self, K):
#         attention
        attention_out = self.multi_head_attention_layer(K, K, K)
        attention_out = self.att_drop(attention_out)
        att_out = self.attention_norm_lay(K + attention_out)
#         feed forward
        linear_out = self.fcn(att_out)
        linear_out = self.linear_drop(linear_out)
        out = self.ff_norm_lay(att_out + linear_out)
        out = att_out + linear_out
    
        return out
class Stack_Decoder(nn.Module):
    """
    Stacked Encoder
    """
    def __init__(self, layer_num, dk, dv, dm, h, p_drop, d_ff):
        super(Stack_Decoder, self).__init__()
        self.decoders = nn.ModuleList([Decoder(dk, dv, dm, h, p_drop, d_ff) for i in range(layer_num)])
        
        
    def forward(self, Q, encoder_out, mask):
        # ModuleList can act as an iterable, or be indexed using ints
        self.mid = Q
        for lay in self.decoders:
            Q = lay(Q, encoder_out, mask=mask)
#             if lay == self.decoders[0]:
#                 self.mid = lay(Q, encoder_out, mask=mask)
        return Q           

class Decoder(nn.Module):
    def __init__(self, dk, dv, dm, h, p_drop, d_ff):
        super(Decoder, self).__init__()
#         query attention residual block
        self.Q_attention_lay = Multi_Head_attention_layer(dk, dv, dm, h)
        self.Q_attention_norm_lay = nn.LayerNorm([dm, ])
        self.Q_att_drop = nn.Dropout(p_drop)
    
#         query key attention residual block
        self.QK_attention_lay = Multi_Head_attention_layer(dk, dv, dm, h)
        self.QK_attention_norm_lay = nn.LayerNorm([dm, ])
        self.QK_att_drop = nn.Dropout(p_drop)
        
    
#         feed forward residual block
        self.fcn = PositionwiseFeedForward(dm, d_ff)
        self.ff_norm_lay = nn.LayerNorm([dm, ])
        self.linear_drop = nn.Dropout(p_drop)
        

    def forward(self, Q, encoder_out, mask):
#         query attention
        Q_attention_out = self.Q_attention_lay(Q, Q, Q, mask=None)
        Q_attention_out = self.Q_att_drop(Q_attention_out)
        Q_att_out = self.Q_attention_norm_lay(Q + Q_attention_out)
#         query key attention
        QK_attention_out = self.QK_attention_lay(Q_att_out, encoder_out, encoder_out, mask=mask)
        QK_attention_out = self.QK_att_drop(QK_attention_out)
        QK_att_out = self.QK_attention_norm_lay(Q_att_out + QK_attention_out)
        
#         feed forward
        linear_out = self.fcn(QK_att_out)
        out = self.ff_norm_lay(QK_att_out + linear_out)
        return out

class Multi_Head_attention_layer(nn.Module):
    def __init__(self, dk, dv, dm, h):
        super(Multi_Head_attention_layer, self).__init__()
        self.Q_linears = nn.ModuleList([nn.Linear(dm, dk) for i in range(h)])
        self.K_linears = nn.ModuleList([nn.Linear(dm, dk) for i in range(h)])
        self.V_linears = nn.ModuleList([nn.Linear(dm, dv) for i in range(h)])
        self.output_linear = nn.Linear(h*dv, dm)
                            

    def forward(self, Q_input, K_input, V_input, mask=None):
        buf = []
        for Q_linear, K_linear, V_linear in zip(self.Q_linears, self.K_linears, self.V_linears):
            Q = Q_linear(Q_input)
            K = K_linear(K_input)
            V = V_linear(V_input)
            buf.append(scaled_dot_attention(Q, K, V, mask))
        
        buf = torch.cat(buf,dim=-1)
        out = self.output_linear(buf)
        
        return out      
class PositionwiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PositionwiseFeedForward, self).__init__()
        self.cnn1 = nn.Conv1d(d_model, d_ff, 1)
        self.cnn2 = nn.Conv1d(d_ff, d_model, 1)
                            

    def forward(self, x):
        bat,seq_len,_ = x.size()
        x = x.permute(0,2,1)
        x = self.cnn1(x)
        x = F.relu(x)
        x = self.cnn2(x)
        x = x.permute(0,2,1)
        
        return x      
    
bat = 3
# Q = torch.rand([bat, 13, D_MODEL]).cuda()
# V = torch.rand([bat, 19, D_MODEL]).cuda()
# Q_mask_len = torch.tensor([2,3,4]).cuda()
# print Q.shape, V.shape, Q_mask_len.shape
# net = Transformer(STACKED_NUM, DK, DV, D_MODEL, H, P_DROP, D_FF).cuda()
# o = net(Q, V, Q_mask_len)
# print(o.size())
# # print o
# def count_parameters(model):
#     return sum(p.numel() for p in model.parameters() if p.requires_grad)
# print(count_parameters(net))



In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
# import Transformer
import numpy as np
from constants import FOOD_NUM
class Net(nn.Module):

    def __init__(self, dm, p_drop):
        super(Net, self).__init__()
        self.drop = nn.Dropout(p_drop)
        self.food_emb = Food_embedding(FOOD_NUM, dm, 2, p_drop)
        self.target_food_emb = nn.Embedding(FOOD_NUM, dm)
        
        self.transformer = Transformer(2, dm, dm, dm, 8, p_drop, d_ff=dm*4)
        
        self.output_linear = nn.Linear(dm, 1)
        self.buf = None

    def forward(self, food, history):
#         print(K.size(), get_pos_mat(MAX_SEQUENCE_LENGTH).size())
        
        Q = food
        K = history
        Q = self.target_food_emb(Q)
        
        K = self.food_emb(K)
        
        
        batch, K_len, d = K.size()
#         Q = self.summary_weight.repeat(batch,1).view(batch, 1, -1)
#         self.buf2 = Q
        
        batch, Q_len, d = Q.size()
        Q_mask_len = np.ones([batch])
        de_out = self.transformer(Q, K, Q_mask_len)
        de_out = de_out.view(batch, d)
        out = self.output_linear(de_out)
        self.mid = out
        
        out = torch.sigmoid(out)
        
        
        return out
class Food_embedding(nn.Module):
    def __init__(self, c_in, dm, layer_num, p_drop, activation_fn=F.selu):
        super(Food_embedding, self).__init__()
        self.activation_fn = activation_fn
        self.drop = nn.Dropout(p_drop)
        assert layer_num >= 1
        self.first_linear = nn.Linear(c_in, dm)
        self.linears = nn.ModuleList([nn.Linear(dm, dm) for i in range(layer_num-1)])
        

    def forward(self, x):
#         print(K.size(), get_pos_mat(MAX_SEQUENCE_LENGTH).size())
        x = self.first_linear(x)
        x = self.activation_fn(x)
        x = self.drop(x)
        for lay in self.linears:
            x = self.drop(self.activation_fn(lay(x)))
        return x
    
batch = 7
dm = 128
K = torch.rand([batch, 18, FOOD_NUM]).cuda()
Q = torch.randint(FOOD_NUM,[batch,1], dtype=torch.long).cuda()
model = Net(dm, 0.1).cuda()
print(Q.dtype)
o = model(Q, K)
# print t
print(o.size())
# print o

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(count_parameters(model))


criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.0001)

In [ ]:
# torch.nn.init.xavier_uniform_(self.summary_weight)
for p in model.parameters():
    if p.requires_grad and p.ndimension() == 2:
        torch.nn.init.xavier_uniform_(p)
#         breakn
        
        

#### prepare sample generator


In [ ]:
from constants import MAX_SEQ_LEN

val_num = len(u_map) // 10
idx = np.random.permutation(len(u_map))
train_idx, val_idx = idx[val_num:], idx[:val_num]
train_u_map = {k:u_map[k] for k in u_map.keys()[val_num:]}
val_u_map = {k:u_map[k] for k in u_map.keys()[:val_num]}
def batch_boostrap_generator(batch_size, u_map, food_map, Y_map, max_history_len):
    G = boostrap_generator(u_map, food_map, Y_map, max_history_len)
    while True:
        X = []
        pos_Y = []
        neg_Y = []
        for i in range(batch_size):
            x, pos_y, neg_y = next(G)
            x = np.pad(x, ((0,max_history_len-x.shape[0]),(0,0)), 'constant', constant_values=0)
            X.append(np.expand_dims(x, axis=0))
            pos_Y.append(np.expand_dims(pos_y, axis=0))
            neg_Y.append(np.expand_dims(neg_y, axis=0))
        yield np.vstack(X), np.vstack(pos_Y), np.vstack(neg_Y) 
def boostrap_generator(u_map, food_map, Y_map, max_history_len):
    while True:
        keys = u_map.keys()
        for user_idx in np.random.permutation(len(u_map)):
            user = keys[user_idx]
            X = np.zeros([max_history_len, len(food_map)])
            Y = Y_map[user].flatten()
            history = u_map[user]
            ds = np.array([d for d,f in history])
            fs = np.array([f for d,f in history])
            ds = ds[np.argsort(ds)]
            fd = fs[np.argsort(ds)]
            
            date_idx = 0
            now_date = ds[0]
            for food, date in zip(fs,ds):
                if date != now_date:
                    date_idx+=1
                    now_date = date
                X[date_idx, food_map[food]] = 1
            
    #         positive sample
            idx = np.random.permutation(len(Y))
            pos_i = neg_i = -1
            for i in idx:
                if Y[i] == 1 and pos_i == -1:
                    pos_i = i
                if Y[i] == 0 and neg_i == -1:
                    neg_i = i
                if pos_i != -1 and neg_i != -1:
                    break
            yield X, np.array([pos_i]), np.array([neg_i])
            
    

G = batch_boostrap_generator(32, train_u_map, food_map, Y_map, max_history_len=MAX_SEQ_LEN)
val_G = batch_boostrap_generator(32//2, val_u_map, food_map, Y_map, max_history_len=MAX_SEQ_LEN)

x, pos_y, neg_y = next(G)
print x.shape, pos_y.shape, neg_y.shape
x, pos_y, neg_y = next(val_G)
print x.shape, pos_y.shape, neg_y.shape

G2 = boostrap_generator(train_u_map, food_map, Y_map, max_history_len=MAX_SEQ_LEN)
x, pos_y, neg_y = next(G2)
print x.shape, pos_y.shape, neg_y.shape


In [ ]:
from collections import deque
from tqdm import tqdm as tqdm

import time
def dump_log(model, n_iter, loss, acc, val_loss, val_acc, log_file_stream, tmp_model_path):
    log_text = '%.7d<split>%.5f<split>%.5f<split>%.5f<split>%.5f\n' % (n_iter, loss, acc, val_loss, val_acc)
    log_file_stream.write(log_text)
    if n_iter % 100 == 0 :
        log_file_stream.flush()
        torch.save(model, tmp_model_path)
        
acc_q = deque(maxlen=1000)
loss_q = deque(maxlen=1000)
val_acc_q = deque(maxlen=1000)
val_loss_q = deque(maxlen=1000)
t = time.time()
best_acc  = 0
best_loss = float('inf')

epochs = 100
batch_size = 32
G = batch_boostrap_generator(batch_size, train_u_map, food_map, Y_map, max_history_len=MAX_SEQ_LEN)
val_G = batch_boostrap_generator(batch_size, val_u_map, food_map, Y_map, max_history_len=MAX_SEQ_LEN)

print 'start training.'
with open('log.txt', 'a') as f:
    for e in range(epochs):
        iters = len(food_map) * len(user_map)
        with tqdm(total=iters) as pbar:
            for it in range(iters):
                optimizer.zero_grad()
                model.train()
                x, pos_y, neg_y = next(G)
#                 print 'x1', np.sum(x[0,:])
                x = torch.FloatTensor(x).cuda()
                pos_y,neg_y = torch.LongTensor(pos_y), torch.LongTensor(neg_y)
                
#                 positive
                output = model(pos_y.cuda(), x)
                a = output[0,0].item()
                pred = output > 0.5
                label = torch.ones_like(output).cuda()
                pos_loss = criterion(output, label)

                acc = torch.sum(pred == label.type(torch.uint8)).item() / float(output.shape[0])
                acc_q.append(acc)
#                 pos_loss.backward()

#                 negative
                output = model(neg_y.cuda(), x)
                b = output[0,0].item()
                pred = output > 0.5
                label = torch.zeros_like(output).cuda()
                neg_loss = criterion(output, label )
                acc = torch.sum(pred == label.type(torch.uint8)).item() / float(output.shape[0])
                acc_q.append(acc)
#                 neg_loss.backward()

                loss = pos_loss + neg_loss
                loss.backward()
                optimizer.step()
                with torch.no_grad():
                    model.eval()
#                 positive
                    x, pos_y, neg_y = next(val_G)
#                     print 'x2', np.sum(x[0,:])
                    x = torch.FloatTensor(x).cuda()
                    pos_y,neg_y = torch.LongTensor(pos_y), torch.LongTensor(neg_y)
#                       
                    output = model(pos_y.cuda(), x)
#                     o1 = model.transformer.decoder.mid
#                     o1 = model.mid
                    
                    c = output[0,0].item()
                    pred = output > 0.5
                    label = torch.ones_like(output).cuda()
                
                    pos_loss = criterion(output, label )
                
                    val_acc = torch.sum(pred == label.type(torch.uint8)).item() / float(output.shape[0])
                    val_acc_q.append(val_acc)
#                 negative
#                     print pos_y, neg_y
#                     print output
                    
                    output = model(neg_y.cuda(), x)
#                     o2 = model.mid
#                     print o2.shape
#                     print neg_y.cuda().shape
#                     print neg_y.cuda()[0,0]
#                     print (o1 == o2).all()
                    d = output[0,0].item()
#                     print c == d
#                     wf
#                     print pos_y == neg_y
#                     print output
                    
    
                    pred = output > 0.5
                    label = torch.zeros_like(output).cuda()
        
                    neg_loss = criterion(output, label )
                    val_acc = torch.sum(pred == label.type(torch.uint8)).item() / float(output.shape[0])
                    val_acc_q.append(val_acc)
                    
                    val_loss = pos_loss + neg_loss
                acc = np.mean(acc_q)
                val_acc = np.mean(val_acc_q)

#                 dump_log(model, it+1, loss, acc, val_loss, val_acc, f, './tmp.pt')

#                 pbar.set_postfix_str('acc : %.3f, val_acc : %.3f, %.3f, %.3f, %.3f' % (acc, val_acc,a,b,c), refresh=False)
                pbar.set_postfix_str('acc : %.3f, val_acc : %.3f, %.3f, %.3f, %.5f, %.5f' % (acc, val_acc,a,b,c,d), refresh=False)
                pbar.update(batch_size)
                dump_log(model, it+1, loss, acc, val_loss, val_acc, f, './tmp.pt')
                if val_acc > best_acc and it > 100:
                    torch.save(model, './best.pt')
                    best_acc = val_acc

# Train model
print("Optimization Finished!")
# print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

  0%|          | 0/14427456 [00:00<?, ?it/s]

start training.


  0%|          | 11520/14427456 [04:50<100:53:20, 39.69it/s, acc : 0.498, val_acc : 0.500, 0.515, 0.482, 0.47901, 0.47901]

In [ ]:
# print acc_q
# print output.shape
# c = output < 0.5
# print output < 0.5

print pred.shape
print label.shape
print torch.sum(pred == label.type(torch.uint8)).item() / float(output.shape[0])

In [ ]:
a = torch.tensor([[0.49]])
b = torch.zeros_like(a)
print a.shape
print a > 0.5
c = a > 0.5
print b,c
print b.type(torch.uint8) == c


In [ ]:
salnj;kvahjk
# print u_map[6]
buf  = [] 
def ck(ds):
    l = max(ds) - min(ds)
    min_d = min(ds)
    
#     d_list = [(d-min_d) for d in ds]
#     d_list = sorted(set(d_list))
    d_list = sorted(set(ds))
#     print l, len(d_list)
    for i,d in enumerate(d_list):
        if i == 0:
            continue
        d_ = d_list[i-1]
        if (d - d_).days.real != 1:
            d_diff = (d - d_).days.real
            buf.append(d_diff)
#             print d_diff
#             assert d_diff < 10
def cf(ds):
    l = max(ds) - min(ds)
    min_d = min(ds)
    
#     d_list = [(d-min_d) for d in ds]
#     d_list = sorted(set(d_list))
    d_list = sorted(set(ds))
    
#     print l, len(d_list)
    for i,d in enumerate(d_list):
        if i == 0:
            continue
        d_ = d_list[i-1]
        if (d - d_).days.real != 1:
            d_diff = (d - d_).days.real
            buf.append(d_diff)
#             print d_diff
#             assert d_diff < 10
                        
    
# for i in [6,8,12]:
with tqdm(total=len(u_map)) as pbar:
    buf = []
    buf2 = []
    for i in u_map.keys():
        ds = [d for d,f in u_map[i]]
        threshold = ((max(ds) - min(ds)).days.real * (4./5.))
        min_d = min(ds)
        fl = []
        m = {}
        for d,f in u_map[i]:
            if (d - min_d).days.real > threshold:
                fl.append(f)
            if d not in m:
                m[d] = []
            m[d].append(f)
        for k,v in m.items():    
            buf2.append(len(set(v)))
#         print len(u_map[i]), len(fl), len(set(fl))
        buf.append(len(set(fl)))
        #     print i, min(ds), max(ds), len(ds)
#         ck(ds)
        pbar.update(1)
# d6 = [d for d,f in u_map[6]]
# ck(d6)
    
#     print ck(ds)
# print u_map[33]
print np.mean(buf), np.std(buf)
print np.mean(buf2), np.std(buf2)


In [ ]:


print min(dates), max(dates),  max(dates)- min(dates)


In [ ]:
print min(dates)
m = min(dates)
print max(dates)
a = dates[0]
print dir(a - min(dates))
print a
# def normalize_date(min_date, max_date, date):
# for k in u_map.keys()[:10]:
#     print len(u_map[k])
ds = [(d-m).total_seconds() / (60*60*24) for d in dates]
# import numpy as np
print np.mean(ds)
print np.min(ds), np.max(ds)

In [ ]:
import keras
from os.path import join
import os
from bs4 import BeautifulSoup as BS
from constants import MAX_SEQUENCE_LENGTH, MAX_NUM_WORDS, EMBEDDING_DIM
from keras.preprocessing.sequence import pad_sequences
import numpy as np
np.random.seed(1337)
def quote_title_abstract(xml_path):
    with open(xml_path, 'r') as f:
        data = f.read()
    soup = BS(data)
    title, abstract = soup.find('title').text, soup.find('abstract').text
    return title.strip(), abstract.strip()

# text preprocessing
data_path = join('./','kaggle/')
xml_dir = join(data_path, 't2-doc')
xml_list = [f for f in os.listdir(xml_dir) if f.endswith('.xml')]
# print(len(xml_list))


texts = []

for xml in xml_list:
    path = join(xml_dir,xml)
    title, abstract = quote_title_abstract(path)
    text = title + '' + abstract
    texts.append(text)
#     texts.append(title)
#     texts.append(abstract)
print('read all %d xml files.' % len(xml_list))
tokenizer = keras.preprocessing.text.Tokenizer(num_words=MAX_NUM_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~ ',
                                   lower=True, split=' ', char_level=False, oov_token=None)
tokenizer.fit_on_texts(texts)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
sequences = tokenizer.texts_to_sequences(texts)
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
xml_id_map = {}
for i,xml in enumerate(xml_list):
    node_id = int(xml.replace('.xml',''))
    xml_id_map[node_id] = data[i,:]


print('Preparing embedding matrix.')
embeddings_index = {}
# with open(os.path.join('./','glove', 'glove.6B.%dd.txt' % EMBEDDING_DIM), 'r', encoding='utf8') as f:
with open(os.path.join('./','glove', 'glove.6B.%dd.txt' % EMBEDDING_DIM), 'r') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
# prepare embedding matrix
num_words = min(MAX_NUM_WORDS, len(word_index)) + 1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i > MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

print('done')